Import all needed libraries

In [109]:
import requests
import urllib.parse
import urllib.request
import json

Set host and API credentials
* __host__               - API host
* __API_CLIENT_ID__      - API client ID
* __API_CLIENT_SECRET__  - API client secret


In [111]:
host = 'https://sandbox.datadryad.org'
API_CLIENT_ID = "[CLIENT_ID]"
API_CLIENT_SECRET = "[CLIENT_SECRET]"

headers = {"Content-Type": "application/json"}

Authorize API connection and retrieve access token

In [112]:
payload = {'client_id': API_CLIENT_ID, 'client_secret': API_CLIENT_SECRET, 'grant_type': 'client_credentials'}
payload = {
    "client_id": API_CLIENT_ID,
    "client_secret": API_CLIENT_SECRET, 
    "grant_type": "client_credentials"
}
url = f'{host}/oauth/token'
response = requests.post(url, data=json.dumps(payload), headers=headers)

if response.status_code != 200:
    raise SystemExit(response.json()['error_description'])
token = response.json()['access_token']

if not token:
    raise SystemExit('Unable to retrieve access token')

headers['Authorization'] = f'{response.json()['token_type']} {token}'

Create dataset

In [115]:
payload = {
    "title": "Data from: Jupyter Notebook uploaded through API",
    "abstract" : "This study develops the random phylogenies rate test (RAPRATE), a likelihood method that simulates morphological evolution along randomly generated phylogenies, and uses it to determine whether a considerable difference in morphological diversity between two sister clades of South American fishes should be taken as evidence of differing rates of morphological change or lineage turnover.",
    "authors" : [ 
        {
            "firstName" : "Test",
    	    "lastName" : "Author",
    	    "email" : "test@examples.com",
    	    "orcid" : "0009-0000-4404-1689",
    	    "affiliation" : "Oregon State University",
    	    "affiliationROR" : "https://ror.org/00ysfqy60"
    	}
    ]
}
headers['Content-Type'] = 'application/json'
url = f'{host}/api/v2/datasets'
response = requests.post(url, data=json.dumps(payload), headers=headers)
dataset_id = response.json()['id']
dataset_url = f'{host}{response.json()['_links']['self']['href']}'

if 'error' in response.json():
    raise SystemExit(response.json()['error']); exit
else:
    print('Resource successfully created')

Resource successfully created


Update dataset and set set al missing info

In [123]:
payload = {
  "title": "A Study of Red-Black Trees with Weight",
  "authors": [
    {
      "firstName": "Matilda",
      "lastName": "Swinton",
      "email": "matilda.swinton@example.com",
      "affiliation": "Example Organization"
    }
  ],
  "abstract": "<p>Information theorists agree that \"fuzzy\" models are an interesting new topic in the field of machine learning.</p>\r\n",
  "funders": [
    {
      "organization": "Stichting Sem Presser Archief",
      "identifierType": "ror",
      "identifier": "https://ror.org/02f5r3008",
      "awardNumber": "12345"
    }
  ],
  "keywords": [
    "Cat",
    "Host",
    "Computer",
    "Log"
  ],
  "methods": "<p><br>\r\nMany mathematicians would agree that, had it not been for systems, the emulation of active networks might never have occurred. We view algorithms as following a cycle of four phases: simulation, investigation, observation, and investigation. After years of robust research into Boolean logic, we confirm the evaluation of Smalltalk. to what extent can spreadsheets be evaluated to realize this intent?</p>\r\n",
  "relatedWorks": [
    {
      "relationship": "article",
      "identifierType": "URL",
      "identifier": "http://example.org/cats"
    }
  ],
  "userId": 2,
  "email": "some_email@example.com"
}

headers['Content-Type'] = 'application/json'
response = requests.put(dataset_url, data=json.dumps(payload), headers=headers)
if response.status_code != 200:
    raise SystemExit(response.json()['error_description'])

Embed a file on the dataset using file 

In [118]:
embed_url = f'{dataset_url}/urls'
payload = {
    "url": "https://github.com/datadryad/dryad-app/blob/e927b514efe97fc08c7fc30d99389a5ec6d8a493/documentation/apis/sample_dataset.json",
    "digest": "aca3032d20c829a6060f1b90afda6d14",
    "digestType": "md5",
    "description": "This is the best file ever!",
    "size": 1234,
    "path": "api_submission.md",
    "mimeType": "text/plain",
    "skipValidation": True
}

headers['Content-Type'] = 'application/json'
response = requests.post(embed_url, data=json.dumps(payload), headers=headers)
if 'error' in response.json():
    raise SystemExit(response.json()['error']); exit
else:
    print('File successfuly embeded')
    
response.json()

SystemExit: You have already supplied this url

Upload a file on the dataset

In [125]:
file_name = 'upload_test.txt'
upload_url = f'{dataset_url}/files/{urllib.parse.quote(file_name, safe='')}'
file_path = f'./{file_name}'

headers['Content-Type'] = 'application/json'
with open(file_path, 'rb') as file:
    files = {'file': file}
    response = requests.put(upload_url, files=files, headers=headers)

if 'error' in response.json():
    raise SystemExit(response.json()['error']); exit
else:
    print('File successfuly uploaded')
    
response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Submit dataset

In [124]:
payload = {
    "op": "replace",
    "path": "/versionStatus",
    "value": "submitted"
}

headers['Content-Type'] = 'application/json-patch+json'
response = requests.patch(dataset_url, data=json.dumps(payload), headers=headers)

errors = [obj['error'] for obj in response.json()]

if len(errors) > 0:
    print('Errors: ')
    print(', '.join(errors))
else:
    print('Status successfully updated')

Errors: 
Fill in submitting author's email, Fill in a research domain, Include a README to describe your dataset.
